## CLIPSPY hands on tutorial

In [1]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

### Data types

CLIPS provides eight primitive data types for representing information. These types are float, integer, symbol, string, external-address, fact-address, instance-name and instance-address. Numeric information can be represented using floats and integers. Symbolic information can be represented using symbols and strings.

A number consists only of digits (0-9), a decimal point (.), a sign (+ or -), and, optionally, an (e) for exponential notation with its corresponding sign. A number is either stored as a float or an integer. Any number consisting of an optional sign followed by only digits is stored as an integer (represented internally by CLIPS as a C long integer). All other numbers are stored as floats (represented internally by CLIPS as a C double-precision float).

In [2]:
## Example floats and integers
env.eval("(+ 3 237e3 -100 0.001)")

236903.001

Symbols are any sequence of characters that starts with any printable ASCII character and is followed by zero or more printable ASCII characters. When a delimiter is found, the symbol is ended. The following characters act as delimiters: any non-printable ASCII character (including spaces, tabs, carriage returns, and line feeds), a double quote, opening and closing parentheses “(” and “)”, an ampersand “&”, a vertical bar “|”, a less than “<”, and a tilde “~”. A semicolon “;” starts a CLIPS comment and also acts as a delimiter.

In [3]:
## Example symbols
env.eval("foo")
env.eval("Hello")
env.eval("B76-HI")
env.eval("2-each")
env.eval("127A")

'127A'

A string is a set of characters that starts with a double quote (") and is followed by zero or more printable characters. A string ends with double quotes. Double quotes may be embedded within a string by placing a backslash (\) in front of the character. A backslash may be embedded by placing two consecutive backslash characters in the string.

In [4]:
## Example strings 
env.eval('"foo"')
env.eval('"a and b"')
env.eval('"1 number"')
env.eval('"a \\"quote\\""')

'a "quote"'

A multifield value is a sequence of zero or more single field values. When displayed by CLIPS, multifield values are enclosed in parentheses. Collectively, single and multifield values are referred to as values.

In [5]:
## Example multifield value
env.eval('(create$ x 3.0 "red" 567)')


['x', 3.0, 'red', 567]

A fact is a list of atomic values that are either referenced positionally (ordered facts) or by name (non-ordered or template facts). Ordered facts looks similar to multifields but are used directly for inference (see below). 

In [6]:
## Example ordered facts
env.assert_string("(the pump is on)")

ImpliedFact: f-1     (the pump is on)

In [7]:
env.assert_string("(grocery-list bread milk eggs)")

ImpliedFact: f-2     (grocery-list bread milk eggs)

Non-ordered facts allow for more structure to be used for representing knowledge. Non-ordered facts can be created using deftemplate or defclass. We will only use deftemplate in this class. 

In [8]:
# Simple template for a person
s = """
 (deftemplate person
    (slot name)
    (slot gender)
    (slot job)
    (slot location)
    (multislot interests))
"""
env.build(s)


In [9]:
# creates an instance of a 'person' fact 

template = env.find_template('person')

new_fact = template.new_fact()
new_fact['name'] = clips.Symbol('Richard')
new_fact['gender'] = clips.Symbol('Male')
new_fact['job'] = clips.Symbol('Professor')
new_fact['location'] = clips.Symbol('Pittsburgh')
new_fact['interests'] = [clips.Symbol('clinical decision support'), clips.Symbol('pharmacovigilance')]
new_fact.assertit()

for fact in env.facts():
    print(fact)

(initial-fact)
(the pump is on)
(grocery-list bread milk eggs)
(person (name Richard) (gender Male) (job Professor) (location Pittsburgh) (interests clinical decision support pharmacovigilance))


In [10]:
# a simpler way to construct a non-ordered fact but does not do any checking for 
# appropriate slot values 
new_fact = env.assert_string("""
 (person 
   (name Harry)
   (gender Male) 
   (job Professor) 
   (location Pittsburgh) 
   (interests human_computer_interaction Education))
)
""")

for fact in env.facts():
    print(fact)

(initial-fact)
(the pump is on)
(grocery-list bread milk eggs)
(person (name Richard) (gender Male) (job Professor) (location Pittsburgh) (interests clinical decision support pharmacovigilance))
(person (name Harry) (gender Male) (job Professor) (location Pittsburgh) (interests human_computer_interaction Education))


#### E1: Exercise with data types

Create a CLIPS template that models demographic information you would collect about a patient during an outpatient visit. Assert three patient facts using the template you created. Be sure to include height (measured in inches) and weight (measured in pounds).

## Functions

Clips has many built in functions and they all look like lists

In [11]:
env.eval("(create$ apple banana peach)")


['apple', 'banana', 'peach']

In [12]:
env.eval("(first$ (create$ a b c))")

['a']

In [13]:
env.eval('(eq 1 1)')

'TRUE'

In [14]:
env.eval('(< 1 2)')

'TRUE'

Functions are evaluated from the inside out

In [15]:
env.eval('(< (+ 1 1)(+ 2 2))')

'TRUE'

In [16]:
env.eval('(eq 4 (+ 2 2))')

'TRUE'

Programmers can define custom functions. The simplest way to do this with clipspy is to write a python function and then use define_function to create the function in the CLIPS environment:

In [17]:
def testFunc(arg):
    print("I am within a Python function, argument: %f" % arg)
    return arg

env.define_function(testFunc)
ret = env.eval('(python-function testFunc 42.2)')
print("Eval returned %f" % ret)

I am within a Python function, argument: 42.200000
Eval returned 42.200000


Programmers can also use CLIPS deffunction to directly create functions 

In [18]:
env.build('''
  (deffunction concat-args (?a ?b ?c)
     (str-cat ?a ", " ?b ", and " ?c))
''')

In [19]:
env.eval('(concat-args One Two Three))')

'One, Two, and Three'

#### E2: Exercise with functions

Write a function to calculate BMI using the pounds and inches version locate at https://www.cdc.gov/healthyweight/assessing/bmi/childrens_bmi/childrens_bmi_formula.html. You can use either CLIPS deffunction or Python define_function to create the new function. We will apply the function to the right hand side (RHS) of rules later.


### Rules

Rules are defined using the defrule construct. 

Redefining a currently existing defrule causes the previous defrule with the same name to be removed even if the new definition has errors in it. 

The left-hand side (LHS) is made up of a series of conditional elements (CEs) that typically consist of pattern conditional elements (or just simply patterns) to be matched against pattern entities. An implicit and conditional element always surrounds all the patterns on the LHS. 

The right-hand side (RHS) contains a list of actions to be performed when the LHS of the rule is satisfied. In addition, the LHS of a rule may also contain declarations about the rule’s properties immediately following the rule’s name and comment. 

The arrow (=>) separates the LHS from the RHS. There is no limit to the number of conditional elements or actions a rule may have (other than the limitation placed by actual available memory). 

Actions are performed sequentially if, and only if, all conditional elements on the LHS are satisfied. If no conditional elements are on the LHS, the rule will automatically be activated. If no actions are on the RHS, the rule can be activated and fired but nothing will happen.

In [20]:
## Example
env.clear()
env.reset()

env.build("""
(deftemplate oav
 (slot object)
 (slot attribute)
 (slot value))
""")

env.build("""
(defrule example-rule
  "This is an example of a simple rule"
  (oav (object refrigerator)
       (attribute light)
       (value on))
  (oav (object refrigerator)
       (attribute door)
       (value open))
  =>
  (assert (oav (object refrigerator)
               (attribute food)
               (value spoiled))))
""")

env.assert_string("""
(oav (object refrigerator)
     (attribute light)
     (value on))
""")

env.assert_string("""
(oav (object refrigerator)
     (attribute door)
     (value open)))
""")

TemplateFact: f-2     (oav (object refrigerator) (attribute door) (value open))

In [21]:
for fact in env.facts():
    print(fact)

(initial-fact)
(oav (object refrigerator) (attribute light) (value on))
(oav (object refrigerator) (attribute door) (value open))


In [22]:
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(oav (object refrigerator) (attribute light) (value on))
(oav (object refrigerator) (attribute door) (value open))
(oav (object refrigerator) (attribute food) (value spoiled))


Another example of simple rules that uses the deffacts construct to assert facts in batch 

In [23]:
env.build("""
(deftemplate bed
    (slot unit)
    (slot bed-number)
    (slot patient-name))
""")

env.build("""
(deftemplate patient
    (slot patient-name)
    (multislot health-status))
""")

In [24]:
env.build("""
(deffacts StartStateOnSlides
    (bed (unit ER)(patient-name Ann))
    (bed (unit ICU)(bed-number 1)(patient-name Barry))
    (bed (unit ICU)(bed-number 2)(patient-name Cleo))
    (bed (unit Unit1)(bed-number 1))
    (bed (unit Unit1)(bed-number 2)(patient-name Darren))
    (bed (unit Unit2)(bed-number 1)(patient-name Eva))
    (bed (unit Unit2)(bed-number 2)(patient-name Frank))
    (patient (patient-name Ann)(health-status very-ill))
    (patient (patient-name Barry)(health-status not-very-ill infectious))
    (patient (patient-name Cleo)(health-status infectious very-ill))
    (patient (patient-name Darren)(health-status not-very-ill))
    (patient (patient-name Eva)(health-status infectious not-very-ill))
    (patient (patient-name Frank)(health-status free2go))
    )
""")

In the next block the 'reset' method loads the facts that were built in the previous blocks into working memory

In [25]:
env.reset()
for fact in env.facts():
    print(fact)

(initial-fact)
(bed (unit ER) (bed-number nil) (patient-name Ann))
(bed (unit ICU) (bed-number 1) (patient-name Barry))
(bed (unit ICU) (bed-number 2) (patient-name Cleo))
(bed (unit Unit1) (bed-number 1) (patient-name nil))
(bed (unit Unit1) (bed-number 2) (patient-name Darren))
(bed (unit Unit2) (bed-number 1) (patient-name Eva))
(bed (unit Unit2) (bed-number 2) (patient-name Frank))
(patient (patient-name Ann) (health-status very-ill))
(patient (patient-name Barry) (health-status not-very-ill infectious))
f-10    (patient (patient-name Cleo) (health-status infectious very-ill))
f-11    (patient (patient-name Darren) (health-status not-very-ill))
f-12    (patient (patient-name Eva) (health-status infectious not-very-ill))
f-13    (patient (patient-name Frank) (health-status free2go))


IMPORTANT NOTE: It is important to always remember the CLIPS runs in an "environment" object created within Python. CLIPS code is executed in the CLIPS environment using Python methods owned by the environment instance. Things are slightly more complicated by the fact that we are working in a Jupyter notebook which means the STDOUT and STDERR don't behave as they do from the command-line or a typical IDE. One of the implications is that we can't use simple print statements to log what is happing as the code runs. Fortunately, we can create a global variable and then append string data to that variable. The next set of blocks show this pattern in action applied to the bed assignment example we went over in class.

In [26]:
# This global variable is where we write statements we want to see printed. We can print the value of the 
# variable out when we want to view it
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')

In [27]:
env.build("""
(defrule R1 "IF ER patient very ill and ICU bed free THEN admit to ICU bed"
    ?patient-fact <- (patient (patient-name ?ptname)(health-status $? very-ill $?))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ER))
    ?newbed-fact <- (bed (patient-name nil)(unit ICU)(bed-number ?newbedno))
    =>
    (modify ?newbed-fact (patient-name ?ptname))
    (retract ?oldbed-fact)
    (bind ?*log* 
        (str-cat ?*log* 
           (format nil "R1 Fired. Patient %s has been moved to ICU bed %d.%n" ?ptname ?newbedno)
        )
    )
)
""")

Notice in the code block above the following:
* variable names can be assigned to pattern matches and used within the RHS of the rule e.g., ?ptname, ?newbedno
* pointers to template instances can be created using <- e.g., ?patient-fact, ?oldbed-fact, ?newbed-fact. These are used in the rule RHS to assert, retract, or modify non-ordered facts 
* retract removed facts from working memory. CLIPS does this similar to the the truth maintenance system we discussed in class where other facts which receive logical support from the retracted fact are also retracted
* pattern matching is used within slots such as health-status that are ordered. In the rule above '$?' is used to match any value to the left or right of 'very ill' in health-status

In [28]:
env.build("""
(defrule R2 "IF there is a patient in an ICU bed who is not very ill and not infectious and there is a Unit 1 bed free THEN transfer to Unit 1 bed"
    (patient (health-status $?status)(patient-name ?ptname))
    (test (not (member$ infectious $?status)))
    (test (member$ not-very-ill $?status))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ICU))
    ?newbed-fact <- (bed (patient-name nil) (unit Unit1)(bed-number ?bedno))
    =>
    (modify ?oldbed-fact (patient-name nil))
    (modify ?newbed-fact (patient-name ?ptname))
    (bind ?*log* 
        (str-cat ?*log*
          (format nil "R2 Fired. Patient %s has been moved to Unit1 bed %d.%n" ?ptname ?bedno)
        )
     )
)
""")

Notice in the code block above the following:

* 'test' is used to check the state of facts that match on the LHS. In this case, each fact that matches 
the generic 'patient' template (which would be all patient instances in working memory) is tested to determine if their status is 'not-very-ill' and that 'infectious' is not present in the status list.

In [29]:

env.build("""
(defrule R3 "If patient is ready for discharge, then free the bed"
    ?patient-fact <- (patient (health-status $? free2go $?)(patient-name ?ptname))
    ?newbed-fact <- (bed (patient-name ?ptname)(unit ?unitname)(bed-number ?bedno))
    =>
    (retract ?patient-fact)
    (modify ?newbed-fact (patient-name nil))
    (bind ?*log* 
        (str-cat ?*log*
	     (format nil "R3 Fired. Patient %s has been discharged, and %s bed %d is now free.%n" ?ptname  ?unitname  ?bedno)
        )
    )
)
""")

env.build("""
(defrule R4 "IF a patient is infectious and not very ill and there is a Unit 2 bed free THEN admit or transfer only to unit 2"
    (patient (health-status $?status)(patient-name ?ptname))
    (test (member$ infectious $?status))
    (test (member$ not-very-ill $?status))
    ?oldbed-fact <- (bed (patient-name ?ptname)(unit ER|ICU)) ; why (unit ER|ICU)?
    ?newbed-fact <- (bed (patient-name nil) (unit Unit2)(bed-number ?bedno))
    =>
    (modify ?oldbed-fact (patient-name nil))
    (modify ?newbed-fact (patient-name ?ptname))
    (bind ?*log* 
        (str-cat ?*log*
           (format nil "R4 Fired. Patient %s has been moved to Unit2 bed %d.%n" ?ptname ?bedno)
        )
     )
)    
""")

Notice in R4 the use of the pipe symbol to match on either 'ER' or 'ICU' in the bed 'unit' slot

In [30]:
env.run()

3

The output tells use how many RHS made changes to working memory

In [31]:
# Here is how we print the value of the global variable so we can see the output of the rule's RHS
v = env.find_global('log')
print(v.value)

# This shows the facts in working memory
for fact in env.facts():
    print(fact)

INFO:
R3 Fired. Patient Frank has been discharged, and Unit2 bed 2 is now free.
R4 Fired. Patient Barry has been moved to Unit2 bed 2.
R1 Fired. Patient Ann has been moved to ICU bed 1.

(initial-fact)
(bed (unit ICU) (bed-number 2) (patient-name Cleo))
(bed (unit Unit1) (bed-number 1) (patient-name nil))
(bed (unit Unit1) (bed-number 2) (patient-name Darren))
(bed (unit Unit2) (bed-number 1) (patient-name Eva))
(patient (patient-name Ann) (health-status very-ill))
(patient (patient-name Barry) (health-status not-very-ill infectious))
f-10    (patient (patient-name Cleo) (health-status infectious very-ill))
f-11    (patient (patient-name Darren) (health-status not-very-ill))
f-12    (patient (patient-name Eva) (health-status infectious not-very-ill))
f-16    (bed (unit Unit2) (bed-number 2) (patient-name Barry))
f-17    (bed (unit ICU) (bed-number 1) (patient-name Ann))


#### E3: Exercise with rules and functions 

Write 4 rules that determine the BMI status of patients using the patient template and BMI function you created. Here are the criteria for the four categories from https://www.nhlbi.nih.gov/health/educational/lose_wt/BMI/bmicalc.htm 

BMI Categories:
Underweight = <18.5
Normal weight = 18.5–24.9
Overweight = 25–29.9
Obesity = BMI of 30 or greater

Create patient facts for each of the 4 categories and show that the patients are properly assigned to a BMI category when running the rule engine. 
